In [1]:
import pandas as pd
import requests
import time
import sys
def pull_data(url):


    headers = {
                                    "Host": "stats.nba.com",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                                    "Accept": "application/json, text/plain, */*",
                                    "Accept-Language": "en-US,en;q=0.5",
                                    "Accept-Encoding": "gzip, deflate, br",

                                    "Connection": "keep-alive",
                                    "Referer": "https://stats.nba.com/"
                                }
    json = requests.get(url,headers = headers).json()
    if len(json["resultSets"])== 1:


        data = json["resultSets"][0]["rowSet"]
        #print(data)
        columns = json["resultSets"][0]["headers"]
        #print(columns)
        
        df = pd.DataFrame.from_records(data, columns=columns)
    else:

        data = json["resultSets"]["rowSet"]
        #print(json)
        columns = json["resultSets"]["headers"][1]['columnNames']
        #print(columns)
        df = pd.DataFrame.from_records(data, columns=columns)
    return df
def get_mincombo(year,ps=False):
    season = str(year-1)+'-'+str(year)[-2:]
    group_count=2
    
    stype ='Regular%20Season'
    trail = " RS"
    if ps == True:
        stype='Playoffs'
        trail= " PS"


    url = 'https://stats.nba.com/stats/leaguedashlineups?Conference=&DateFrom=&DateTo=&Division=&GameSegment=&GroupQuantity='+str(group_count)+'&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType='+stype+'&ShotClockRange=&TeamID=0&VsConference=&VsDivision='
    df = pull_data(url)
    df=df.sort_values(by='MIN',ascending=False)
    index_frame=pd.read_csv('player_index.csv',low_memory=False)
    year=year
    year_frame=index_frame[index_frame.year==year]
    nba_ids=year_frame.PLAYER_ID.astype(str).tolist()
    
    player_combos=[]
    
    for nba_id in nba_ids:
        player_df=df[df.GROUP_ID.str.contains(nba_id)]
        player_df=player_df.head(5)
        min=player_df['MIN'].tolist()
   
        series1=player_df['GROUP_ID'].str.replace(nba_id, '', regex=False)
        series1=series1.str.replace('-','').tolist()
        player_df=pd.DataFrame()
  
        count=1

        for min,teammate in zip(min,series1):
            player_df['min'+str(count)]=[min]
            player_df['teammate'+str(count)]=[teammate]
            count+=1
        
        player_df['nba_id']=nba_id
        player_df['year']=year


        player_combos.append(player_df)

   

  
        
    return pd.concat(player_combos)
def all_years(ps=False):
    frames=[]
    for year in range(2008,2025):
        df = get_mincombo(year,ps=ps)
        frames.append(df)
        print(year)
    return pd.concat(frames)
rs_combo=all_years()
rs_combo.to_csv('rs_combo.csv',index=False)
ps_combo=all_years(ps=True)

ps_combo.to_csv('ps_combo.csv',index=False)

2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
